# 前処理

In [2]:
#pandasの準備
import pandas as pd
#データ読み込み[1]
review = pd.read_csv("amazon_レビュー詳細.csv")#Octoparse[2]というスクレイピングツール内のテンプレートを活用し収集しました
print(len(review))

13324


In [3]:
#欠損値を確認
#レビュータイトルなし、役に立った人数が0人だと空欄(=欠損値)になっている
review.isna().sum()

ページUrl           0
ASIN             0
Id               0
投稿日              0
星                0
名前               4
レビュー_タイトル    13324
レビュー             0
役に立った         3606
dtype: int64

In [4]:
#欠損値をなくす(エラーになるから)
#参考(https://note.nkmk.me/python-pandas-nan-fillna/)

#Nan = "なし"と置き換える
review= review.fillna("なし")

#欠損値を再度確認する
review.isna().sum()

ページUrl       0
ASIN         0
Id           0
投稿日          0
星            0
名前           0
レビュー_タイトル    0
レビュー         0
役に立った        0
dtype: int64

In [5]:
#星の部分をfloat型に変換する
#数値のデータにしておかないと後でエラーになる(該当部分で説明します)

for i in range(len(review)):
    s = review["星"].iloc[i]#レビュー毎に星部分を参照
    s = s[6:]#「5つ星のうち3.0」->「3.0」
    s = float(s)#小数点を含むのでintではなくfloat
    review["星"].iloc[i] = s#レビュー毎にデータを置き換える

In [6]:
#mecab[3]の準備
#mecab：掲載素解析に使うモジュール
!pip install mecab-python3 unidic-lite

import MeCab
tagger = MeCab.Tagger()

# 辞書(IPA)ver.

In [7]:
#目的：全レビューにおける名詞の登場回数と平均評価を集計する

#助詞「の」「で」を抜く(以前実行した際に名詞に含まれてしまっていた為)
stop_words = ["の", "で"]
all_words = []
parts = ["名詞"]
star = []#星を入れるリストの準備


for n in range(len(review)):
    text = review["レビュー"].iloc[n]#1つずつレビューを指定(for文で回る)
    words = tagger.parse(text).splitlines()
    words_arr = []
    mini_star = []
    for i in words:
        if i == "EOS" or i == "": continue#1文解析ごとにに表示するEOSまたは空白なら終了
        word_tmp = i.split()[0]#レビューの単語区切り要素
        if len(i.split()) >= 4:#単語が品詞要素を持つなら進む
          part = i.split()[4].split("-")[0]
          if not (part in parts):continue#名詞でなければリストに入れない
          for i in range(len(stop_words)):
            if word_tmp in stop_words[i]:continue#stop_wordsにいたら入れない
          if not(word_tmp in words_arr):
            words_arr.append(word_tmp)
          s = review["星"].iloc[n]#単語に対応するレビューの星部分を参照
          if not(len(words_arr) == len(mini_star)):
            mini_star.append(s)#↑を「star」に加える
    all_words.extend(words_arr)
    star.extend(mini_star)
  
#words_arr，mini_star->レビューごとの名詞or評価リスト
#all_words,star->全レビューの名詞or評価単語リスト(↑をどんどんくっつけている)

In [8]:
all_words_df = pd.DataFrame({"words":all_words, "star":star, "count":len(all_words)*[1]})#それぞれの単語にcount1を追加する
words_star = all_words_df.groupby("words").mean()["star"]#単語毎にstarの平均値を求める
words_count = all_words_df.groupby("words").sum()["count"]#単語毎にcountの合計値を求める
words_df = pd.concat([words_star, words_count], axis = 1)#axis=1：列方向で

#↑をcountの多い順に並び替える
df_star = words_df.sort_values("count", ascending=False)
df_star.head(20)

,star,count
words,,
ゲーム,3.594781,4215
プレイ,3.669511,2578
こと,3.510328,2469
購入,4.079393,2305
ストーリー,3.630106,2179
時間,3.611203,1821
方,3.448832,1798
人,3.534046,1777
2,3.422440,1631


In [9]:
#count上位100個をstarの高い順に並び替える
df_star_100 = df_star.head(100)
df_star_100 = df_star_100.sort_values("star", ascending=False)

In [10]:
#結果をcsvで出力
df_star_100.to_csv("①ipa_登場回数top100_高評価順.csv")

# 辞書(Neologd)ver.

In [11]:
'''''
固有名詞はゲーム内容を構成する際に不適切なのではないかと考えた．(ex:ポケ◯ンが良い単語ならポケ◯ン売ればいいじゃん！となる)
固有名詞を除去するという項目を追加し，再度単語を集計する
辞書を現代語に強いものに変更する(固有名詞をより正しく固有名詞と判定しやすい)[4]
'''''

tagger = MeCab.Tagger("-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd")

In [12]:
#目的：前レビューの名詞リスト「all_words」を作成する


#辞書の特性上，ひらがな一文字が登場しがちだが不要なのでstop_wordsとする
stop_words = ["で", "あ","い","う","え","お","か","き","く","け","こ","さ","し","す","せ","そ","た","ち","つ","て","と","な","に","ぬ","ね","の","は","ひ","ふ","へ","ほ","ま","み","む","め","も","や","ゆ","よ","ら","り","る","れ","ろ","わ","を","ん"]
all_words_koyunasi = []
parts = ["名詞"]
star = []

#形態素解析をし、名詞に該当する単語をall_wordsというリストに格納していく(for文でまわる)
for n in range(len(review)):
    text = review["レビュー"].iloc[n]#1つずつレビューを指定(for文で回る)
    words = tagger.parse(text).splitlines()
    words_arr = []
    mini_star = []
    for i in words:
        if i == "EOS" or i == "": continue#1文解析ごとにEOSと表示される、空白
        try:
          errorhakuyo = i.split()[1]
        except IndexError:
          continue
        word_tmp = i.split()[0]#レビューの単語区切り要素
        if word_tmp in stop_words:continue#「stop_words」にいたら入れない
        #print(i.split())
        word_hinshi = i.split()[1]
        word_hinshi = word_hinshi.split(",")
        part = word_hinshi[0]#単語ごとの品詞　#名詞-普通名詞-一般
        if not (part in parts):continue#名詞でなければリストに入れない
        if word_hinshi[1] == "固有名詞":continue
        if word_hinshi[1] == "数":continue
        if not(word_tmp in words_arr):
          words_arr.append(word_tmp)#↑にひっかからなかった要素が「words_arr」に追加される
        s = review["星"].iloc[n]#単語に対応するレビューの星部分を参照
        if not(len(words_arr) == len(mini_star)):
          mini_star.append(s)#↑を「star」に加える

    all_words_koyunasi.extend(words_arr)#1レビュー分終わったら、「all_words」に「words_arr」をくっつける
    star.extend(mini_star)


#words_arr，mini_star->レビューごとの名詞or評価リスト
#all_words_koyunasi,star->全レビューの名詞or評価単語リスト(↑をどんどんくっつけている)

In [13]:
#ipa辞書ver.と同様の作業をする
all_words_koyunasi_df = pd.DataFrame({"words":all_words_koyunasi, "star":star, "count":len(all_words_koyunasi)*[1]})


words_star_k = all_words_koyunasi_df.groupby("words").mean()["star"]
words_count_k = all_words_koyunasi_df.groupby("words").sum()["count"]
words_df_k = pd.concat([words_star_k, words_count_k], axis = 1)

#words_df_kをcountの多い順に並び替える
df_star_k = words_df_k.sort_values("count", ascending=False)

In [14]:
#ipa辞書ver.と同様の作業をする
#加えて，5段階評価それぞれの個数も別途数えておく(kazu_mitai_k)
all_words_koyunasi_df = pd.DataFrame({"words":all_words_koyunasi, "star":star, "count":len(all_words_koyunasi)*[1]})

kazu_mitai_k = {}
for i in range(len(all_words_koyunasi_df)):
    if not(all_words_koyunasi_df.iloc[i,0] in kazu_mitai_k):
        kazu_mitai_k[all_words_koyunasi_df.iloc[i,0]] = [all_words_koyunasi_df.iloc[i,1]]
    else:
        kazu_mitai_k[all_words_koyunasi_df.iloc[i,0]].append(all_words_koyunasi_df.iloc[i,1])

In [15]:
import statistics

hosi_1 = []
jyogaishitai = []

for i in range(len(kazu_mitai_k)):
     a = all_words_koyunasi_df.iloc[i,0]
     hosi_2 = [a,kazu_mitai_k[a].count(1.0),kazu_mitai_k[a].count(2.0),kazu_mitai_k[a].count(3.0),kazu_mitai_k[a].count(4.0),kazu_mitai_k[a].count(5.0),statistics.mean(kazu_mitai_k[a]),len(kazu_mitai_k[a])]
     if not(a in jyogaishitai):
          jyogaishitai.append(a)
          hosi_1.append(hosi_2)

hosi_3 = sorted(hosi_1,key=lambda x:x[7],reverse=True)
print(type(hosi_3))
df_list2_k = pd.DataFrame(hosi_3,columns=["words","1.0","2.0","3.0","4.0","5.0","star","count"])

<class 'list'>


In [16]:
df_list2_k.sort_values("count", ascending=False)
tango = df_list2_k.head(100)
tango.to_csv("②neologd_登場回数top100_高評価順.csv", encoding="Shift-JIS")

In [17]:
tango.head(20)

,words,1.0,2.0,3.0,4.0,5.0,star,count
0,ゲーム,632,382,476,668,1589,3.587136,3747
1,こと,419,272,340,493,934,3.508950,2458
2,よう,351,262,401,484,887,3.542558,2385
3,購入,218,136,192,419,1310,4.084396,2275
4,ストーリー,217,283,361,498,791,3.633953,2150
5,方,315,222,291,373,653,3.446063,1854
6,人,249,175,283,329,602,3.525031,1638
7,これ,276,192,231,264,521,3.378706,1484
8,キャラ,186,206,265,304,418,3.407542,1379
9,戦闘,148,211,264,303,415,3.466816,1341


# 頻出単語を含むレビューの頻出単語表

In [18]:
#自然言語処理ではよく係り受けやその可視化がされている
#頻出単語にどういう言葉がくっついてるかなっていうのを見ている
#その関係性の高さとか，登場回数が多かったりすると関係性が強いですよって出力される
#じゃあ，頻出単語から高評価の単語を抽出して，それを含むレビューにおける頻出単語表を作れば↑みたいな事ができるのでは？

tango

,words,1.0,2.0,3.0,4.0,5.0,star,count
0,ゲーム,632,382,476,668,1589,3.587136,3747
1,こと,419,272,340,493,934,3.508950,2458
2,よう,351,262,401,484,887,3.542558,2385
3,購入,218,136,192,419,1310,4.084396,2275
4,ストーリー,217,283,361,498,791,3.633953,2150
...,...,...,...,...,...,...,...,...
95,フィールド,41,62,76,108,140,3.571429,427
96,自体,71,63,76,123,87,3.219048,420
97,不満,40,39,78,120,143,3.683333,420
98,画面,61,61,81,98,117,3.356459,418


In [19]:
#良い意味を持つ単語を(★4) + (★5)が6割以上と定義し，該当する単語を抽出する
tango_good = []
for i in range(len(tango)):
    hosi4=tango.iloc[i,4]
    hosi5=tango.iloc[i,5]
    zentai=tango.iloc[i,7]
    if (hosi4+hosi5)/zentai >=0.6:
        tango_good.append(tango.iloc[i,0])

In [20]:
#もしレビューがtango_goodを含んでいたらdaizi_reviewに入れる
daiji_review = []
for n in range(len(review)):
    text = review["レビュー"].iloc[n]#1つずつレビューを指定(for文で回る)
    for j in range(len(tango_good)):
        key = tango_good[j]
        if key in text:
            if not (text in daiji_review):
                daiji_review.append(text)

In [21]:
#リストの長さ
print(len(daiji_review))
#正しくレビューを抽出できているか確認する
print(len(list(dict.fromkeys(daiji_review))))

8724
8724


In [22]:
#目的：daiji_reviewから名詞リスト「all_words」を作成する
#殆どneologdのところに書いたものと同じ．


stop_words = ["で", "あ","い","う","え","お","か","き","く","け","こ","さ","し","す","せ","そ","た","ち","つ","て","と","な","に","ぬ","ね","の","は","ひ","ふ","へ","ほ","ま","み","む","め","も","や","ゆ","よ","ら","り","る","れ","ろ","わ","を","ん"]
all_words_next = []
parts = ["名詞"]


for n in range(len(daiji_review)):
    text = daiji_review[n]
    words = tagger.parse(text).splitlines()
    words_arr = []
    mini_star = []
    for i in words:
        if i == "EOS" or i == "": continue
        try:#エラーが出てもそれ以下の実行を続ける
          errorhakuyo = i.split()[1]
        except IndexError:
          continue

        word_tmp = i.split()[0]
        if word_tmp in stop_words:continue
        #print(i.split())
        word_hinshi = i.split()[1]
        word_hinshi = word_hinshi.split(",")
        part = word_hinshi[0]
        if not (part in parts):continue
        if word_hinshi[1] == "固有名詞":continue
        if word_hinshi[1] == "数":continue
        if not(word_tmp in words_arr):
          words_arr.append(word_tmp)


    all_words_next.extend(words_arr)


In [23]:
all_words_next_df = pd.DataFrame({"words":all_words_next, "count":len(all_words_next)*[1]})
all_words_next_df = all_words_next_df.groupby("words").sum()

#↑をcountの多い順に並び替える
words_count_next = all_words_next_df.sort_values("count", ascending=False)

In [24]:
words_count_next.head(20)

,count
words,
ゲーム,3318
こと,2083
購入,2075
よう,2060
ストーリー,1830
方,1571
人,1420
これ,1261
キャラ,1209


In [25]:
words_count_next.to_csv("③好評価な単語の頻出単語表.csv")

# 参考文献

[1]Amazon(Amazon.com, Inc.)2023-01-29   
https://www.amazon.co.jp/   
[2]Octoparseスクレイピング|Webクローラー(Octopus Data Inc.)2023-01-29   
https://www.octoparse.jp/   
[3]MeCab(工藤拓)2023-01-29   
https://taku910.github.io/mecab/   
[4]mecab-ipadic-NEologd : Neologism dictionary for MeCab(Toshinori Sato)2023-01-29   
https://github.com/neologd/mecab-ipadic-neologd/blob/master/README.ja.md   